# CalCOFI
### Over 60 years of oceanographic data
#### Dataset: https://www.kaggle.com/datasets/sohier/calcofi
#### Table Info: https://calcofi.org/data/oceanographic-data/bottle-database/

## EDA and Data Preparation

#### Load Data

In [ ]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAll([
    ('spark.master', 'local[1]'), 
    ('spark.app.name', 'App Name')])
    
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.version

In [ ]:
bottle = spark.read.csv("file:///home/work/Final/data/bottle.csv", header=True).cache()
bottle.count()

In [ ]:
cast = spark.read.csv("file:///home/work/Final/data/cast.csv", header=True).cache()
cast.count()

In [ ]:
bottle.printSchema()

In [ ]:
cast.printSchema()

In [ ]:
bottle.show(2)

In [ ]:
cast.show(2)

#### Remove columns with low data count

#### Remove/Fill Null Values

#### Remove columns that are not needed

#### Handle quality values

#### Make sure column values are the right type

#### Summary Statistics and graphs

## Questions

#### Which factors are chlorophyll levels most dependent on?
#### Can we use these factors as features in a model that accurately predict chlorophyll levels in a marine ecosystem?
#### Can these few, simple, measurable factors we found indicate whether an ocean ecosystem is healthy and sustainable based on the chlorophyll abundance measured?
#### Can we predict chlorophyll abundance without having to measure it?

## Feature Selection and Target Cleaning

#### Distribution of data for cholorophyll levels, summary stats/graphs

#### Define cholorophyll category levels: low, medium, high?

#### Map cholorophyll levels to defined categories and output graphs/summary stats

#### Add target column with category label to dataframe

#### PCA columns that are duplicates

#### Heat Map to see any less obvious columns that are highly correlated

#### If necessary, lasso to find the most important 10 features (columns)

#### Select features to be used for training

## Models

#### Scale Data

#### Split data into 80/10/10 train, test, validation set

### Logistic Regresssion

#### Cross Validation

#### Check to see if data can be used to predict the lables

#### Train and Test Accuracy Scores

### AdaBoost Decision Tree

#### Can we increase accuracy from previous steps? Can we figure out which features matter the most?

#### Train and Test Accuracy Scores

#### Kernel SVM

## Data Visualization

#### SVM Decision Boundary Graph

#### AdaBoost Decision Tree Graph
https://www.ashishmenkudale.com/spark-tree-plotting/